# Simple and Complex Cells in Visual Cortex

## Introduction

In this notebook, we'll explore two fundamental types of neurons in the primary visual cortex (V1): simple cells and complex cells. These neurons were first characterized by David Hubel and Torsten Wiesel in their Nobel Prize-winning work in the 1960s and represent key computational elements in early visual processing.

We'll examine:
- The receptive field properties of simple and complex cells
- How these cells respond to visual stimuli
- The concept of phase sensitivity versus phase invariance
- Quadrature pairs and their role in modeling complex cells
- How complex cells likely combine simple cell outputs
- The hierarchical organization of visual processing

Let's start by setting up our environment and importing necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from scipy import signal
import ipywidgets as widgets
from IPython.display import display, HTML

# Set plotting parameters for better visualization
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12

## 1. Simple Cells: Receptive Fields and Response Properties

Simple cells in V1 respond to specific oriented edges or bars in particular locations within their receptive fields. Their key properties include:

- Elongated receptive fields with specific orientation preference
- Distinct ON and OFF subregions
- Linear summation of inputs
- Position (phase) specific responses

Let's visualize a simple cell receptive field using a Gabor filter, which provides a good approximation of simple cell receptive fields.

In [ ]:
def gabor_filter(size, lambda_val, theta, sigma, gamma, phi):
    """
    Create a Gabor filter.
    
    Parameters:
    - size: Size of the filter (pixels)
    - lambda_val: Wavelength of the sinusoidal component
    - theta: Orientation (radians)
    - sigma: Standard deviation of the Gaussian envelope
    - gamma: Spatial aspect ratio
    - phi: Phase offset (radians)
    
    Returns:
    - 2D Gabor filter
    """
    # Generate coordinate grids
    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    # Rotate coordinates
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)
    
    # Calculate Gabor filter
    gb = np.exp(-(x_theta**2 + (gamma * y_theta)**2) / (2 * sigma**2)) * np.cos(2 * np.pi * x_theta / lambda_val + phi)
    
    return gb

# Create and visualize a simple cell receptive field using a Gabor filter
simple_rf = gabor_filter(size=64, lambda_val=10, theta=np.pi/4, sigma=8, gamma=0.5, phi=0)

plt.figure(figsize=(10, 8))
plt.imshow(simple_rf, cmap='gray')
plt.colorbar(label='Response')
plt.title('Simple Cell Receptive Field (Gabor Filter)')
plt.axis('off')
plt.show()

# Create interactive widgets to explore parameters
@widgets.interact(
    orientation=widgets.FloatSlider(min=0, max=np.pi, step=np.pi/12, value=np.pi/4, description='Orientation'),
    wavelength=widgets.IntSlider(min=5, max=20, step=1, value=10, description='Wavelength'),
    phase=widgets.FloatSlider(min=0, max=2*np.pi, step=np.pi/6, value=0, description='Phase'),
    sigma=widgets.FloatSlider(min=4, max=16, step=1, value=8, description='Sigma'),
    gamma=widgets.FloatSlider(min=0.2, max=1.0, step=0.1, value=0.5, description='Aspect Ratio')
)
def update_gabor(orientation, wavelength, phase, sigma, gamma):
    rf = gabor_filter(size=64, lambda_val=wavelength, theta=orientation, sigma=sigma, gamma=gamma, phi=phase)
    plt.figure(figsize=(8, 6))
    plt.imshow(rf, cmap='gray')
    plt.colorbar(label='Response')
    plt.title('Simple Cell Receptive Field (Gabor Filter)')
    plt.axis('off')
    plt.show()

### Simple Cell Responses to Stimuli

Simple cells respond strongly when a stimulus (like a bar or edge) aligns with their receptive field pattern. They show linear spatial summation, with excitatory and inhibitory subregions that correspond to the ON and OFF regions of the receptive field.

Let's demonstrate simple cell responses to different stimuli.

In [ ]:
def create_bar_stimulus(size, orientation, position, width=3):
    """
    Create a bar stimulus with given orientation and position.
    
    Parameters:
    - size: Size of the image (pixels)
    - orientation: Orientation of the bar (radians)
    - position: Position of the bar center from the image center
    - width: Width of the bar
    
    Returns:
    - 2D image with a bar
    """
    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    # Rotate coordinates
    x_rot = x * np.cos(orientation) + y * np.sin(orientation)
    
    # Create bar at specified position
    bar = np.zeros((size, size))
    bar[np.abs(x_rot - position) < width / 2] = 1
    
    return bar

def simulate_simple_cell_response(stimulus, receptive_field):
    """
    Simulate simple cell response to a stimulus by convolving with receptive field.
    
    Parameters:
    - stimulus: 2D stimulus image
    - receptive_field: 2D receptive field
    
    Returns:
    - Response value (dot product of stimulus and receptive field)
    """
    # Normalize the receptive field
    rf_normalized = receptive_field / np.sqrt(np.sum(receptive_field**2))
    
    # Calculate response (dot product)
    response = np.sum(stimulus * rf_normalized)
    
    return response

# Create a simple cell receptive field
orientation = np.pi/4  # 45 degrees
simple_rf = gabor_filter(size=64, lambda_val=10, theta=orientation, sigma=8, gamma=0.5, phi=0)

# Create interactive widget to demonstrate phase sensitivity
@widgets.interact(
    position=widgets.FloatSlider(min=-20, max=20, step=1, value=0, description='Position')
)
def show_simple_cell_phase_sensitivity(position):
    # Create bar stimulus at different positions
    stimulus = create_bar_stimulus(size=64, orientation=orientation, position=position)
    
    # Calculate response
    response = simulate_simple_cell_response(stimulus, simple_rf)
    
    # Visualize
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    # Display stimulus
    axs[0].imshow(stimulus, cmap='gray')
    axs[0].set_title('Stimulus (Bar)')
    axs[0].axis('off')
    
    # Display receptive field
    axs[1].imshow(simple_rf, cmap='gray')
    axs[1].set_title('Simple Cell Receptive Field')
    axs[1].axis('off')
    
    # Display response
    axs[2].bar(0, response, width=0.5)
    axs[2].set_xlim(-0.5, 0.5)
    axs[2].set_ylim(-1, 1)
    axs[2].set_title(f'Response: {response:.2f}')
    axs[2].set_xticks([])
    axs[2].set_ylabel('Response')
    
    plt.tight_layout()
    plt.show()

### Phase Sensitivity of Simple Cells

As demonstrated above, simple cells show strong phase sensitivity. The response of a simple cell varies significantly with the exact position (phase) of the stimulus within its receptive field. When the bright regions of the stimulus align with the ON regions of the receptive field and the dark regions align with the OFF regions, the cell responds maximally. When this alignment is reversed, the cell may be inhibited.

This phase sensitivity means that simple cells are precise about the exact position of features within their receptive fields.

## 2. Complex Cells: Phase Invariance and Response Properties

Complex cells, also found in V1, share some properties with simple cells but have distinctive differences:

- They respond to oriented edges/bars regardless of exact position within their receptive field
- They lack clearly defined ON/OFF subregions
- They exhibit nonlinear spatial summation
- They often respond to movement in a particular direction
- They demonstrate phase invariance

Let's explore how complex cells achieve phase invariance and model their responses.

In [ ]:
def simulate_complex_cell(stimulus, gabor_even, gabor_odd):
    """
    Simulate complex cell response using a quadrature pair model.
    
    Parameters:
    - stimulus: 2D stimulus image
    - gabor_even: Even-phase Gabor filter (cosine)
    - gabor_odd: Odd-phase Gabor filter (sine)
    
    Returns:
    - Complex cell response
    """
    # Normalize filters
    gabor_even_norm = gabor_even / np.sqrt(np.sum(gabor_even**2))
    gabor_odd_norm = gabor_odd / np.sqrt(np.sum(gabor_odd**2))
    
    # Calculate simple cell responses
    response_even = np.sum(stimulus * gabor_even_norm)
    response_odd = np.sum(stimulus * gabor_odd_norm)
    
    # Complex cell response is the squared sum
    complex_response = response_even**2 + response_odd**2
    
    return complex_response, response_even, response_odd

# Create a quadrature pair of Gabor filters
orientation = np.pi/4  # 45 degrees
gabor_even = gabor_filter(size=64, lambda_val=10, theta=orientation, sigma=8, gamma=0.5, phi=0)  # Cosine phase (even)
gabor_odd = gabor_filter(size=64, lambda_val=10, theta=orientation, sigma=8, gamma=0.5, phi=np.pi/2)  # Sine phase (odd)

# Create interactive widget to demonstrate phase invariance
@widgets.interact(
    position=widgets.FloatSlider(min=-20, max=20, step=1, value=0, description='Position')
)
def show_complex_cell_phase_invariance(position):
    # Create bar stimulus at different positions
    stimulus = create_bar_stimulus(size=64, orientation=orientation, position=position)
    
    # Calculate responses
    complex_response, even_response, odd_response = simulate_complex_cell(stimulus, gabor_even, gabor_odd)
    
    # Visualize
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    
    # Top row: Stimulus and filters
    axs[0, 0].imshow(stimulus, cmap='gray')
    axs[0, 0].set_title('Stimulus (Bar)')
    axs[0, 0].axis('off')
    
    axs[0, 1].imshow(gabor_even, cmap='gray')
    axs[0, 1].set_title('Even Gabor Filter (cos)')
    axs[0, 1].axis('off')
    
    axs[0, 2].imshow(gabor_odd, cmap='gray')
    axs[0, 2].set_title('Odd Gabor Filter (sin)')
    axs[0, 2].axis('off')
    
    # Bottom row: Responses
    axs[1, 0].bar(0, even_response, width=0.5)
    axs[1, 0].set_xlim(-0.5, 0.5)
    axs[1, 0].set_ylim(-1, 1)
    axs[1, 0].set_title(f'Even Response: {even_response:.2f}')
    axs[1, 0].set_xticks([])
    
    axs[1, 1].bar(0, odd_response, width=0.5)
    axs[1, 1].set_xlim(-0.5, 0.5)
    axs[1, 1].set_ylim(-1, 1)
    axs[1, 1].set_title(f'Odd Response: {odd_response:.2f}')
    axs[1, 1].set_xticks([])
    
    axs[1, 2].bar(0, complex_response, width=0.5, color='red')
    axs[1, 2].set_xlim(-0.5, 0.5)
    axs[1, 2].set_ylim(0, 1)
    axs[1, 2].set_title(f'Complex Cell Response: {complex_response:.2f}')
    axs[1, 2].set_xticks([])
    
    plt.tight_layout()
    plt.show()

## 3. Quadrature Pairs and Phase Invariance

A key concept in understanding complex cells is the quadrature pair. A quadrature pair consists of two filters with identical spatial frequency and orientation but a 90° phase difference.

In our model above, we used:
- An even-symmetric Gabor filter (cosine phase, φ = 0)
- An odd-symmetric Gabor filter (sine phase, φ = π/2)

By combining the squared responses of these two filters, we achieve phase invariance. This model represents a plausible mechanism for how complex cells operate.

Let's visualize the complete response profile of a complex cell across different stimulus positions and compare it to simple cells.

In [ ]:
# Calculate responses across a range of positions
positions = np.arange(-20, 21, 1)
simple_even_responses = []
simple_odd_responses = []
complex_responses = []

for pos in positions:
    stimulus = create_bar_stimulus(size=64, orientation=orientation, position=pos)
    complex_resp, even_resp, odd_resp = simulate_complex_cell(stimulus, gabor_even, gabor_odd)
    
    simple_even_responses.append(even_resp)
    simple_odd_responses.append(odd_resp)
    complex_responses.append(complex_resp)

# Plot responses
plt.figure(figsize=(12, 6))

plt.plot(positions, simple_even_responses, 'b-', label='Simple Cell (Even)')
plt.plot(positions, simple_odd_responses, 'g-', label='Simple Cell (Odd)')
plt.plot(positions, complex_responses, 'r-', linewidth=2, label='Complex Cell')

plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Bar Position')
plt.ylabel('Response')
plt.title('Simple vs. Complex Cell Responses')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Analysis of Phase Invariance

The graph above clearly demonstrates the phase invariance property of complex cells:

- **Simple cells** (blue and green lines) show strong modulation with position/phase. They respond positively at some positions and negatively at others. Their response crosses zero multiple times, indicating complete cancellation at certain positions.

- **Complex cells** (red line) maintain a positive response across all positions where the bar is within the receptive field. The response profile shows much less modulation with position, demonstrating phase invariance.

This phase invariance makes complex cells well-suited for detecting features regardless of their exact position within the receptive field, making them more robust feature detectors.

## 4. Mathematical Basis for Phase Invariance

Let's take a closer look at the mathematical basis for phase invariance in complex cells.

For a sinusoidal grating stimulus with spatial frequency $f$ and phase $\phi_s$, the responses of our quadrature pair filters are:

Even filter response: $R_{even} \propto \cos(\phi_s)$

Odd filter response: $R_{odd} \propto \sin(\phi_s)$

The complex cell response is:

$R_{complex} = R_{even}^2 + R_{odd}^2 \propto \cos^2(\phi_s) + \sin^2(\phi_s) = 1$

By the Pythagorean identity, this sum equals 1 regardless of the phase $\phi_s$, resulting in phase invariance.

Let's visualize this concept using the unit circle.

In [ ]:
# Create a phase invariance demonstration using the unit circle
phases = np.linspace(0, 2*np.pi, 100)
cos_resp = np.cos(phases)
sin_resp = np.sin(phases)
complex_resp = cos_resp**2 + sin_resp**2  # Should be 1 for all phases

# Create interactive visualization
@widgets.interact(
    phase=widgets.FloatSlider(min=0, max=2*np.pi, step=0.1, value=0, description='Phase')
)
def show_phase_invariance_unit_circle(phase):
    cos_val = np.cos(phase)
    sin_val = np.sin(phase)
    complex_val = cos_val**2 + sin_val**2
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    
    # Unit circle visualization
    circle = plt.Circle((0, 0), 1, fill=False, color='black')
    axs[0].add_patch(circle)
    axs[0].plot([0, cos_val], [0, sin_val], 'r-', linewidth=2)
    axs[0].plot([0, cos_val], [0, 0], 'b--', label=f'cos(φ) = {cos_val:.2f}')
    axs[0].plot([cos_val, cos_val], [0, sin_val], 'g--', label=f'sin(φ) = {sin_val:.2f}')
    
    axs[0].set_xlim(-1.5, 1.5)
    axs[0].set_ylim(-1.5, 1.5)
    axs[0].grid(True)
    axs[0].set_aspect('equal')
    axs[0].set_xlabel('cos(φ)')
    axs[0].set_ylabel('sin(φ)')
    axs[0].set_title('Unit Circle Representation')
    axs[0].legend()
    
    # Response visualization
    axs[1].plot(phases, cos_resp**2, 'b-', label='$cos^2(φ)$')
    axs[1].plot(phases, sin_resp**2, 'g-', label='$sin^2(φ)$')
    axs[1].plot(phases, complex_resp, 'r-', linewidth=2, label='$cos^2(φ) + sin^2(φ)$')
    
    axs[1].axvline(x=phase, color='k', linestyle='--')
    axs[1].plot(phase, cos_val**2, 'bo', markersize=8)
    axs[1].plot(phase, sin_val**2, 'go', markersize=8)
    axs[1].plot(phase, complex_val, 'ro', markersize=8)
    
    axs[1].set_xlim(0, 2*np.pi)
    axs[1].set_ylim(0, 1.2)
    axs[1].set_xlabel('Phase (φ)')
    axs[1].set_ylabel('Response')
    axs[1].set_title('Responses vs. Phase')
    axs[1].legend()
    axs[1].grid(True)
    
    plt.tight_layout()
    plt.show()

## 5. Hierarchical Processing: From Simple to Complex Cells

The visual system processes information in a hierarchical manner, with each level extracting increasingly complex features from the visual input:

1. **Retina**: Detects local light intensity changes
2. **LGN**: Maintains center-surround organization with enhanced contrast sensitivity
3. **V1 Simple Cells**: Detect oriented edges at specific positions
4. **V1 Complex Cells**: Detect oriented features regardless of exact position
5. **Higher Visual Areas**: Extract more complex features like motion, shape, etc.

This hierarchy allows the visual system to build increasingly abstract representations of the visual scene. Complex cells, by achieving position invariance, represent an important step in this abstraction process.

Let's visualize this hierarchical organization.

In [ ]:
# Create a visual representation of the hierarchical processing
def center_surround_filter(size, sigma_center=2, sigma_surround=4):
    """
    Create a center-surround receptive field (like LGN).
    """
    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    center = np.exp(-(x**2 + y**2) / (2 * sigma_center**2))
    surround = np.exp(-(x**2 + y**2) / (2 * sigma_surround**2))
    
    # Normalize to zero sum
    center = center / np.sum(center)
    surround = surround / np.sum(surround)
    
    return center - surround

# Create a natural image for demonstration
def natural_image(size=64):
    """
    Create a simple natural image with edges.
    """
    img = np.zeros((size, size))
    
    # Add some features
    img[20:40, 10:30] = 1.0  # Square
    img[10:50, 40:45] = 0.7  # Vertical bar
    
    # Add noise and blur
    img += np.random.normal(0, 0.05, (size, size))
    img = np.clip(img, 0, 1)
    img = signal.gaussian_filter(img, sigma=1)
    
    return img

# Create input and filters
input_image = natural_image(size=64)
lgn_filter = center_surround_filter(size=32)
simple_filter_even = gabor_filter(size=32, lambda_val=8, theta=np.pi/4, sigma=4, gamma=0.5, phi=0)
simple_filter_odd = gabor_filter(size=32, lambda_val=8, theta=np.pi/4, sigma=4, gamma=0.5, phi=np.pi/2)

# Compute responses
lgn_response = signal.convolve2d(input_image, lgn_filter, mode='same')
simple_response_even = signal.convolve2d(input_image, simple_filter_even, mode='same')
simple_response_odd = signal.convolve2d(input_image, simple_filter_odd, mode='same')
complex_response = np.sqrt(simple_response_even**2 + simple_response_odd**2)  # Energy response

# Visualize
fig, axs = plt.subplots(2, 4, figsize=(16, 8))

# Input and filters (top row)
axs[0, 0].imshow(input_image, cmap='gray')
axs[0, 0].set_title('Input Image')
axs[0, 0].axis('off')

axs[0, 1].imshow(lgn_filter, cmap='gray')
axs[0, 1].set_title('LGN Filter')
axs[0, 1].axis('off')

axs[0, 2].imshow(simple_filter_even, cmap='gray')
axs[0, 2].set_title('Simple Cell Filter (Even)')
axs[0, 2].axis('off')

axs[0, 3].imshow(simple_filter_odd, cmap='gray')
axs[0, 3].set_title('Simple Cell Filter (Odd)')
axs[0, 3].axis('off')

# Responses (bottom row)
axs[1, 0].text(0.5, 0.5, "Retina", ha='center', va='center', fontsize=14)
axs[1, 0].axis('off')

axs[1, 1].imshow(np.abs(lgn_response), cmap='inferno')
axs[1, 1].set_title('LGN Response')
axs[1, 1].axis('off')

axs[1, 2].imshow(np.abs(simple_response_even), cmap='inferno')
axs[1, 2].set_title('Simple Cell Response')
axs[1, 2].axis('off')

axs[1, 3].imshow(complex_response, cmap='inferno')
axs[1, 3].set_title('Complex Cell Response')
axs[1, 3].axis('off')

plt.tight_layout()
plt.show()

# Add arrows to show the hierarchical flow
plt.figure(figsize=(12, 3))
plt.axis('off')
plt.text(0.1, 0.5, "Retina", ha='center', va='center', fontsize=14)
plt.text(0.3, 0.5, "LGN", ha='center', va='center', fontsize=14)
plt.text(0.5, 0.5, "V1 Simple Cells", ha='center', va='center', fontsize=14)
plt.text(0.7, 0.5, "V1 Complex Cells", ha='center', va='center', fontsize=14)
plt.text(0.9, 0.5, "Higher Visual Areas", ha='center', va='center', fontsize=14)

# Add arrows
plt.arrow(0.15, 0.5, 0.08, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')
plt.arrow(0.35, 0.5, 0.08, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')
plt.arrow(0.58, 0.5, 0.08, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')
plt.arrow(0.78, 0.5, 0.08, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')

plt.title('Hierarchical Visual Processing')
plt.tight_layout()
plt.show()

## 6. Implications for Motion Processing

The distinction between simple and complex cells has important implications for motion processing:

- **Simple cells** can signal local oriented features (like edges) but their phase sensitivity makes them suboptimal for tracking features over time as they move.

- **Complex cells**, with their phase invariance, can continue to respond to a feature as it moves within their receptive field, making them better suited for motion detection.

This functional difference is one reason why complex cells often show direction selectivity and contribute to motion processing pathways.

In the next module (04_v1_motion), we'll explore how these properties enable direction selectivity and motion detection in V1 neurons.

## 7. Summary

In this notebook, we explored the key properties and differences between simple and complex cells in the visual cortex:

- **Simple cells**:
  - Have distinct ON/OFF subregions
  - Show linear spatial summation
  - Exhibit phase sensitivity (position-dependent responses)
  - Can be modeled with Gabor filters

- **Complex cells**:
  - Lack clearly defined ON/OFF subregions
  - Show nonlinear spatial summation
  - Exhibit phase invariance (position-independent responses)
  - Can be modeled using quadrature pairs of simple cells

We demonstrated how combining responses from simple cells with different phases (quadrature pairs) can produce the phase invariance characteristic of complex cells. This computational model, where complex cell responses are computed as the sum of squared outputs from a pair of simple cells, provides a plausible mechanism for how the visual system achieves this invariance.

Phase invariance is a critical step in the hierarchical processing of visual information, allowing the visual system to detect features regardless of their exact position. This property is essential for more complex visual tasks, including motion perception, which we'll explore in upcoming modules.